In [60]:
# packages
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import TimeSeriesSplit
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf


In [2]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/curtailment_target_features.csv', sep = ';', index_col=0)

Imputing missing values

In [43]:
# Loop through DataFrame rows
for index, row in df.iterrows():
    # Check if column 'forecast_solar_MW' is NaN
    if pd.isna(row['forecast_solar_MW']):
        # If 'forecast_solar_MW' is NaN, fill it with the value from 'actual_solar_MW'
        df.at[index, 'forecast_solar_MW'] = row['actual_solar_MW']

    if pd.isna(row['actual_solar_MW']):
      df.at[index, 'actual_solar_MW'] = row['forecast_solar_MW']


columns_to_interpolate = ["wind_speed_m/s",  "wind_direction_degrees", "humidity_percent", "radiation_global_J/m2", "air_temperature_K", "wind_gust_max_m/s", "wind_direction_gust_max_degrees", "forecast_solar_MW", "actual_solar_MW", "total_grid_load_MWh", "residual_load_MWh", "pumped_storage_MWh"]

# Assuming df is your DataFrame with missing values
df[columns_to_interpolate] = df[columns_to_interpolate].interpolate(method='linear', limit_direction='both')

Dropping level column and converting index to datetime type

---



In [44]:
df.drop('level', inplace=True, axis=1)

df.index = pd.to_datetime(df.index)

Dropping actual solar and wind speed to reduce multicollinearity

In [45]:
df.drop(['wind_speed_m/s', 'actual_solar_MW'], inplace=True, axis=1)

In [47]:
df.head()

,redispatch,wind_direction_degrees,radiation_global_J/m2,air_temperature_K,humidity_percent,wind_gust_max_m/s,wind_direction_gust_max_degrees,forecast_solar_MW,total_grid_load_MWh,residual_load_MWh,pumped_storage_MWh
timestamp,,,,,,,,,,,
2020-01-01 00:00:00,0.0,250.0,0.0,278.45,80.70,3.30,250.0,0.0,730.95,624.18,3.25
2020-01-01 00:15:00,0.0,265.0,0.0,278.55,79.95,4.25,265.0,0.0,727.23,620.78,4.35
2020-01-01 00:30:00,0.0,240.0,0.0,278.45,80.40,3.50,250.0,0.0,722.20,611.37,6.40
2020-01-01 00:45:00,0.0,250.0,0.0,278.55,79.45,3.80,245.0,0.0,719.22,604.33,8.32
2020-01-01 01:00:00,0.0,260.0,0.0,278.45,80.50,3.90,260.0,0.0,717.07,600.83,6.32


Introducing time lagging for features

In [48]:
df_lagged = pd.DataFrame(index=df.index)

for feature in df.columns: # dependent variable included!
    df_lagged[feature] = df[feature]
    df_lagged[feature + '_lag1'] = df[feature].shift(1)
    df_lagged[feature + '_lag2'] = df[feature].shift(2)
    df_lagged[feature + '_lag3'] = df[feature].shift(3)

df_lagged.dropna(inplace = True) # maybe better ways
df_lagged.drop(['redispatch_lag1', 'redispatch_lag2', 'redispatch_lag3'], axis=1, inplace = True)

df_lagged

,redispatch,wind_direction_degrees,wind_direction_degrees_lag1,wind_direction_degrees_lag2,wind_direction_degrees_lag3,radiation_global_J/m2,radiation_global_J/m2_lag1,radiation_global_J/m2_lag2,radiation_global_J/m2_lag3,air_temperature_K,...,total_grid_load_MWh_lag2,total_grid_load_MWh_lag3,residual_load_MWh,residual_load_MWh_lag1,residual_load_MWh_lag2,residual_load_MWh_lag3,pumped_storage_MWh,pumped_storage_MWh_lag1,pumped_storage_MWh_lag2,pumped_storage_MWh_lag3
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:45:00,0.0,250.0,240.0,265.0,250.0,0.0,0.0,0.0,0.0,278.55,...,727.23,730.95,604.33,611.37,620.78,624.18,8.32,6.40,4.35,3.25
2020-01-01 01:00:00,0.0,260.0,250.0,240.0,265.0,0.0,0.0,0.0,0.0,278.45,...,722.20,727.23,600.83,604.33,611.37,620.78,6.32,8.32,6.40,4.35
2020-01-01 01:15:00,0.0,260.0,260.0,250.0,240.0,0.0,0.0,0.0,0.0,278.75,...,719.22,722.20,598.70,600.83,604.33,611.37,8.55,6.32,8.32,6.40
2020-01-01 01:30:00,0.0,250.0,260.0,260.0,250.0,0.0,0.0,0.0,0.0,278.55,...,717.07,719.22,590.62,598.70,600.83,604.33,9.12,8.55,6.32,8.32
2020-01-01 01:45:00,0.0,255.0,250.0,260.0,260.0,0.0,0.0,0.0,0.0,278.55,...,713.63,717.07,582.72,590.62,598.70,600.83,14.73,9.12,8.55,6.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-30 22:45:00,0.0,165.0,160.0,165.0,180.0,0.0,0.0,0.0,0.0,279.35,...,735.43,747.10,359.37,369.53,380.90,395.40,13.00,5.63,0.42,1.23
2023-12-30 23:00:00,0.0,170.0,165.0,160.0,165.0,0.0,0.0,0.0,0.0,279.55,...,725.25,735.43,336.22,359.37,369.53,380.90,5.98,13.00,5.63,0.42
2023-12-30 23:15:00,0.0,180.0,170.0,165.0,160.0,0.0,0.0,0.0,0.0,279.65,...,713.42,725.25,319.40,336.22,359.37,369.53,13.38,5.98,13.00,5.63


Adding weekday/weekend columns and season columns

In [52]:
# Assuming df_lagged is your DataFrame with a datetime index
# Create a new column for weekday/weekend
df_lagged['weekday'] = df_lagged.index.weekday < 5

# Create a new column for season based on month
df_lagged['season'] = (df_lagged.index.month % 12 + 3) // 3

# Perform one-hot encoding for the 'season' column
df_lagged = pd.get_dummies(df_lagged, columns=['season'], drop_first=True)

# Perform one-hot encoding for the 'weekday' column
df_lagged = pd.get_dummies(df_lagged, columns=['weekday'], drop_first=True)

# Convert the 'weekday' and 'season' columns to numerical
df_lagged['weekday'] = df_lagged['weekday_True'].astype(int)
df_lagged['season'] = df_lagged[['season_2', 'season_3', 'season_4']].idxmax(axis=1).str.extract(r'(\d)').astype(int)

# Drop the intermediate columns created during one-hot encoding
df_lagged.drop(columns=['weekday_True', 'season_2', 'season_3', 'season_4'], inplace=True)

# Now df_lagged contains the original data with the new weekday and season columns encoded numerically


In [54]:
df_lagged.head()

,redispatch,wind_direction_degrees,wind_direction_degrees_lag1,wind_direction_degrees_lag2,wind_direction_degrees_lag3,radiation_global_J/m2,radiation_global_J/m2_lag1,radiation_global_J/m2_lag2,radiation_global_J/m2_lag3,air_temperature_K,...,residual_load_MWh,residual_load_MWh_lag1,residual_load_MWh_lag2,residual_load_MWh_lag3,pumped_storage_MWh,pumped_storage_MWh_lag1,pumped_storage_MWh_lag2,pumped_storage_MWh_lag3,weekday,season
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:45:00,0.0,250.0,240.0,265.0,250.0,0.0,0.0,0.0,0.0,278.55,...,604.33,611.37,620.78,624.18,8.32,6.40,4.35,3.25,1,2
2020-01-01 01:00:00,0.0,260.0,250.0,240.0,265.0,0.0,0.0,0.0,0.0,278.45,...,600.83,604.33,611.37,620.78,6.32,8.32,6.40,4.35,1,2
2020-01-01 01:15:00,0.0,260.0,260.0,250.0,240.0,0.0,0.0,0.0,0.0,278.75,...,598.70,600.83,604.33,611.37,8.55,6.32,8.32,6.40,1,2
2020-01-01 01:30:00,0.0,250.0,260.0,260.0,250.0,0.0,0.0,0.0,0.0,278.55,...,590.62,598.70,600.83,604.33,9.12,8.55,6.32,8.32,1,2
2020-01-01 01:45:00,0.0,255.0,250.0,260.0,260.0,0.0,0.0,0.0,0.0,278.55,...,582.72,590.62,598.70,600.83,14.73,9.12,8.55,6.32,1,2


Extra trees classifier

In [55]:
start_time = '2021-12-31 23:45:00'

df_lagged = df_lagged[df_lagged.index > start_time]

In [59]:
# Extract features (X) and target variable (y)
X = df_lagged.drop(columns=['redispatch'])
y = df_lagged['redispatch']

# Initialize TimeSeriesSplit with the desired number of splits
tscv = TimeSeriesSplit(n_splits=5)

# Initialize lists to store train and test indices
train_indices = []
test_indices = []

# Perform Time Series Cross Validation
for train_index, test_index in tscv.split(X):
    train_indices.append(train_index)
    test_indices.append(test_index)

# Choose the desired split for train and test sets
split_index = 4  # choose the last split to have 80% data in train set and 20% in the test set

# Get the train and test indices
train_index = train_indices[split_index]
test_index = test_indices[split_index]

# Split the dataframe into train and test sets
train_X, train_y = X.iloc[train_index], y.iloc[train_index]
test_X, test_y = X.iloc[test_index], y.iloc[test_index]

# Print the lengths of train and test sets
print("Train set length:", len(train_X))
print("Test set length:", len(test_X))

# Initialize the Extra Trees Classifier
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, max_features='sqrt', min_samples_leaf=4, min_samples_split=2, max_depth=20, random_state=42)

# Initialize SMOTE for class imbalance handling
smote = SMOTE(random_state=42)

# Initialize StandardScaler for feature scaling
scaler = StandardScaler()

print("Applying SMOTE for class imbalance handling...")
# Apply SMOTE for class imbalance handling
train_X_resampled, train_y_resampled = smote.fit_resample(train_X, train_y)

print("Scaling the features...")
# Scale the features
train_X_scaled = scaler.fit_transform(train_X_resampled)
test_X_scaled = scaler.transform(test_X)

print("Training the Extra Trees Classifier...")
# Fit the model to the training data
extra_trees_clf.fit(train_X_scaled, train_y_resampled)

print("Making predictions on train and test sets...")
# Make predictions on the train and test data
train_y_pred = extra_trees_clf.predict(train_X_scaled)
test_y_pred = extra_trees_clf.predict(test_X_scaled)

print("Calculating evaluation metrics for train set...")
# Calculate evaluation metrics for train set
train_f1 = f1_score(train_y_resampled, train_y_pred, average='macro')
train_precision = precision_score(train_y_resampled, train_y_pred, average='macro', zero_division='warn')

print("Calculating evaluation metrics for test set...")
# Calculate evaluation metrics for test set
test_f1 = f1_score(test_y, test_y_pred, average='macro')
test_precision = precision_score(test_y, test_y_pred, average='macro', zero_division='warn')

# Print average scores across all folds for train and test sets
print("Train F1 score:", train_f1)
print("Train precision score:", train_precision)
print("Test F1 score:", test_f1)
print("Test precision score:", test_precision)


Train set length: 58340
Test set length: 11668
Applying SMOTE for class imbalance handling...
Scaling the features...
Training the Extra Trees Classifier...
Making predictions on train and test sets...
Calculating evaluation metrics for train set...
Calculating evaluation metrics for test set...
Train F1 score: 0.9796146786370097
Train precision score: 0.9798145915903719
Test F1 score: 0.5018879240540509
Test precision score: 0.5307973523914371


Saving the extra trees model

In [61]:
# Save Extra Trees classifier
joblib.dump(extra_trees_clf, 'extra_trees_classifier.pkl')

['extra_trees_classifier.pkl']

1. first test of extra trees classifier with 100 estimators and with no time lagging (2020 - end of 2023)
  
  Average F1 score: 0.48574702158730904
  Average precision score: 0.5065032114306256

2. second test of extra trees classifier with 500 estimators and with no time lagging (2020 - end of 2023)

  Average F1 score: 0.4860819102848346
  Average precision score: 0.5197146774933564

3. third test of extra trees classifier with 100 estimators and with time lagging - the features are also scaled and class imbalance is solved (2020 - end of 2023)

  Average F1 score: 0.5022292421470077
  Average precision score: 0.5135108579330385

4. fourth test of extra trees classifier with 500 estimators and with time lagging - the features are also scaled and class imbalance is solved (2020 - end of 2023)

  Average F1 score: 0.5022939357149807
  Average precision score: 0.5139316629373413

5. fifth test of extra trees classifier with 500 estimators and with time lagging - the features are also scaled and class imbalance is solved (2020 - end of 2023) - NO WIND SPEED AND ACTUAL SOLAR
  
  Average F1 score: 0.5032077844561853
  Average precision score: 0.5142210693172059

6. sixth test of extra trees classifier with 500 estimators and with time lagging - the features are also scaled and class imbalance is solved (2020 - end of 2023) - NO WIND GUST MAX AND FORECAST SOLAR

  Average F1 score: 0.5038018272452696
  Average precision score: 0.5219906097438674

7. seventh test of extra trees classifier with 100 estimators and with time lagging - the features are also scaled and class imbalance is solved (2020 - end of 2023) - NO WIND GUST MAX AND ACTUAL SOLAR
  
  Average Train F1 score: 0.9877561977704226
  Average Train precision score: 0.9881442108759486
  Average Test F1 score: 0.5060695859875237
  Average Test precision score: 0.5218312260860379

8. eigth test of extra trees classifier with 100 estimators and with time lagging - the features are also scaled and class imbalance is solved (2020 - end of 2023) - NO WIND SPEED AND ACTUAL SOLAR

  Average Train F1 score: 0.987464728765999
  Average Train precision score: 0.9878657689591807
  Average Test F1 score: 0.507415861963602
  Average Test precision score: 0.5253941436935031

9. ninth test of extra trees classifier with 100 estimators and with time lagging - the features are also scaled and class imbalance is solved (2022 - end of 2023) - NO WIND SPEED AND ACTUAL SOLAR

  Train F1 score: 0.9706662734715747
  Train precision score: 0.9713627820195279
  Test F1 score: 0.5074936981415743
  Test precision score: 0.5278601341308067

10. tenth test of extra trees classifier with 100 estimators and with time lagging - the features are also scaled and class imbalance is solved (2022 - end of 2023) - NO WIND SPEED AND ACTUAL SOLAR & WEEKDAY + SEASON COLS ADDED

  Train F1 score: 0.9796146786370097
  Train precision score: 0.9798145915903719
  Test F1 score: 0.5018879240540509
  Test precision score: 0.5307973523914371

Trying extra trees classifier with a grid search to find the best hyperparameter set

In [28]:
start_time = '2021-12-31 23:45:00'

df_lagged = df_lagged[df_lagged.index > start_time]

#data is reduced to only 2 years - 2022 and 2023

In [29]:
# Initialize TimeSeriesSplit with the desired number of splits
tscv = TimeSeriesSplit(n_splits=5)

# Initialize lists to store train and test indices
train_indices = []
test_indices = []

# Perform Time Series Cross Validation
for train_index, test_index in tscv.split(df_lagged.index):
    train_indices.append(train_index)
    test_indices.append(test_index)

# Choose the desired split for train and test sets
split_index = 4  # choose the last split to have 80% data in train set and 20% in the test set

# Get the train and test indices
train_index = train_indices[split_index]
test_index = test_indices[split_index]

# Split the dataframe into train and test sets
train_df = df_lagged.iloc[train_index]
test_df = df_lagged.iloc[test_index]

# Print the lengths of train and test sets
print("Train set length:", len(train_df))
print("Test set length:", len(test_df))


Train set length: 58340
Test set length: 11668


,redispatch,wind_speed_m/s,wind_speed_m/s_lag1,wind_speed_m/s_lag2,wind_speed_m/s_lag3,wind_direction_degrees,wind_direction_degrees_lag1,wind_direction_degrees_lag2,wind_direction_degrees_lag3,radiation_global_J/m2,...,total_grid_load_MWh_lag2,total_grid_load_MWh_lag3,residual_load_MWh,residual_load_MWh_lag1,residual_load_MWh_lag2,residual_load_MWh_lag3,pumped_storage_MWh,pumped_storage_MWh_lag1,pumped_storage_MWh_lag2,pumped_storage_MWh_lag3
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0.0,4.80,5.15,4.80,4.75,270.0,270.0,270.0,270.0,0.0,...,757.32,764.58,212.98,215.08,223.97,230.88,39.98,50.40,46.20,47.45
2022-01-01 00:15:00,0.0,4.80,4.80,5.15,4.80,265.0,270.0,270.0,270.0,0.0,...,747.00,757.32,209.07,212.98,215.08,223.97,39.67,39.98,50.40,46.20
2022-01-01 00:30:00,0.0,5.70,4.80,4.80,5.15,270.0,265.0,270.0,270.0,0.0,...,744.10,747.00,208.60,209.07,212.98,215.08,43.30,39.67,39.98,50.40
2022-01-01 00:45:00,0.0,6.25,5.70,4.80,4.80,270.0,270.0,265.0,270.0,0.0,...,737.25,744.10,199.50,208.60,209.07,212.98,46.33,43.30,39.67,39.98
2022-01-01 01:00:00,0.0,6.20,6.25,5.70,4.80,270.0,270.0,270.0,265.0,0.0,...,728.78,737.25,194.30,199.50,208.60,209.07,39.12,46.33,43.30,39.67


In [33]:
# Prepare the data
X_train = train_df.drop(columns=['redispatch'])  # Extract features
y_train = train_df['redispatch']  # Extract target

X_test = test_df.drop(columns=['redispatch'])  # Extract features
y_test = test_df['redispatch']  # Extract target

In [38]:
# Initialize the Extra Trees Classifier
extra_trees_clf = ExtraTreesClassifier(random_state=42)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Initialize GridSearchCV with the classifier and parameter grid
grid_search = GridSearchCV(extra_trees_clf, param_grid, cv=tscv, scoring='f1_macro', verbose=2)

# Perform grid search using the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best F1 score:", best_score)

# Evaluate the best model on the testing set
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_test)
f1_test = f1_score(y_test, y_pred, average='macro')
precision_test = precision_score(y_test, y_pred, average='macro', zero_division='warn')
print("F1 score on the testing set:", f1_test)
print("Precision score on the testing set:", precision_test)


Fitting 5 folds for each of 540 candidates, totalling 2700 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.5s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.5s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   4.6s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   5.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=

Best Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Best F1 score: 0.5167404206535176
F1 score on the testing set: 0.5237467789810708
Precision score on the testing set: 0.5584318606693672